# Transformers Interpret NER Example

### Setup

In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

c:\Users\pablo\DFKI\transformers-interpret\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import NER model

In [2]:
model_path = 'dslim/bert-base-NER'
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

We can explored the possible entity tags that this model classifies each token into:

In [3]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}


Import the **TokenClassificationExplainer** from the *transformers interpret* package. 

In [18]:
from transformers_interpret import TokenClassificationExplainer
from pprint import pprint

Create an instance of the explainer providing the model and the tokenizer

In [6]:
ner_explainer = TokenClassificationExplainer(model=model, tokenizer=tokenizer)

### Basic usage

Call the explainer to generate the attributions. In the NER task, for each token of the sentence, the predicted class is considered as the target and the attributions of the rest of the tokens in the sentence are calculated. This can take up to one or two minutes. In the following sections we will see how to limit the computation of the attributions to only certain tokens or only certain NER labels.

In [7]:
sample_text = "We visited Paris last weekend, where Emmanuel Macron lives."

In [8]:
attributions = ner_explainer(sample_text)

The explainer returns a list of tuples in which the first position is one of the tokens of text and the second position are the attribtuions. The `predicted_class_names` property provides the predicted classes of the last input provided to the explainer.

In [19]:
index = 3

token, attr = attributions[index]
predicted_class = ner_explainer.predicted_class_names[index]

print(f"Atributions for token '{token}', classified as '{predicted_class}':")
pprint(attr)

Atributions for token 'Paris', classified as 'B-LOC':
[('[CLS]', 0.0),
 ('We', -0.026176158026416003),
 ('visited', 0.1289424214695528),
 ('Paris', 0.9820248557100411),
 ('last', -0.13107116600328042),
 ('weekend', 0.010904143358261256),
 (',', 0.011178508562829472),
 ('where', 0.023509729193194558),
 ('Emmanuel', 0.009830564279598433),
 ('Mac', 0.00034993661827931806),
 ('##ron', 0.014679480802688051),
 ('lives', 0.00523146911109651),
 ('.', 9.093164937093907e-05),
 ('[SEP]', 0.0)]


**Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

####  Visualizing explanations

## Visualizating Explanations 
With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to classifying each of the tokens into a particular class. 

In [21]:
html = ner_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
O,O (1.00),[CLS],1.13,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),We,0.16,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),visited,1.15,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-LOC,B-LOC (1.00),Paris,1.03,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),last,1.46,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),weekend,1.97,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),",",2.23,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),where,1.94,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-PER,B-PER (1.00),Emmanuel,1.17,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),Mac,1.27,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"


#### Ignore indexes

To save computation time, we can indicate a list of token indexes that we want to ignore. The explainer will not compute explanations for these tokens, although attributions of these tokens will calculated to explain the predictions over other tokens.

In [22]:
ner_explainer_2 = TokenClassificationExplainer(model=model, tokenizer=tokenizer, ignored_indexes=[0,1,2,14])

In [ ]:
attributions_2 = ner_explainer_2(sample_text)